This notebook is to combine all the State that provide Covid case count during 2020 to 2021 

In [1]:
# Dependencies and Setup
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np  

In [2]:
# Path to the directory containing your CSV files
school_path = 'school_case/'

# Use glob to get a list of file paths matching a specific pattern (e.g., all CSV files)
school_csv = glob.glob(school_path + '*.csv')

# Initialize an empty list to store individual DataFrames
list_of_dfs = []

# Loop through each CSV file, read it into a DataFrame, and append it to the list
for file in school_csv:
    df = pd.read_csv(file, encoding='utf-8')
    list_of_dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(list_of_dfs, ignore_index=True)

C:\Users\vivi4\AppData\Local\Temp\ipykernel_15208\4186341828.py:12: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding='utf-8')


In [3]:
# Define columns to keep
schoolCase_df = combined_df[['StateName', 'StateAbbrev', 'NCESDistrictID', 'TimePeriodEnd', 'NewCasesStudents', 'CumulativeCasesStudents' ]]

In [4]:
# Remove rows where 'NCESDistrictID' and both 'NewCasesStudents' and 'CumulativeCasesStudents' columns are empty
schoolCase_df = schoolCase_df.dropna(subset=['NCESDistrictID'], how='all').dropna(subset=['NewCasesStudents', 'CumulativeCasesStudents'], how='all')

schoolCase_df.count()

StateName                  60115
StateAbbrev                60115
NCESDistrictID             60115
TimePeriodEnd              60115
NewCasesStudents           20619
CumulativeCasesStudents    45420
dtype: int64

In [5]:
# Verify data types
schoolCase_df.dtypes

StateName                   object
StateAbbrev                 object
NCESDistrictID             float64
TimePeriodEnd               object
NewCasesStudents           float64
CumulativeCasesStudents     object
dtype: object

In [6]:
# Replace NaN and NA values in 'CumulativeCasesStudents' with 0
schoolCase_df['CumulativeCasesStudents'].fillna(0, inplace=True)

# Convert 'CumulativeCasesStudents' to string to handle NA values
schoolCase_df['CumulativeCasesStudents'] = schoolCase_df['CumulativeCasesStudents'].astype(str)

# Create a boolean mask for rows with numeric patterns in 'CumulativeCasesStudents'
numeric_rows = schoolCase_df['CumulativeCasesStudents'].str.match(r'^\d+(\.\d+)?$', na=False)

# Filter the DataFrame using the numeric_rows mask
schoolCase_df = schoolCase_df[numeric_rows].copy()

# Convert 'CumulativeCasesStudents' to float
schoolCase_df['CumulativeCasesStudents'] = schoolCase_df['CumulativeCasesStudents'].astype(float).astype(int)
schoolCase_df['NewCasesStudents'] = schoolCase_df['NewCasesStudents'].astype(pd.Int64Dtype())
schoolCase_df.dtypes

StateName                   object
StateAbbrev                 object
NCESDistrictID             float64
TimePeriodEnd               object
NewCasesStudents             Int64
CumulativeCasesStudents      int32
dtype: object

In [7]:
# Sort the DataFrame by 'StateName', 'NCESDistrictID', and 'TimePeriodEnd'
schoolCase_df.sort_values(by=['StateName', 'NCESDistrictID', 'TimePeriodEnd'], inplace=True)

schoolCase_df['NCESDistrictID'] = schoolCase_df['NCESDistrictID'].astype(str).str.replace('.', '')

# Convert 'TimePeriodEnd' to datetime
schoolCase_df['TimePeriodEnd'] = pd.to_datetime(schoolCase_df['TimePeriodEnd'])

# Reset the index
schoolCase_df.reset_index(drop=True, inplace=True)
schoolCase_df.count()

StateName                  59972
StateAbbrev                59972
NCESDistrictID             59972
TimePeriodEnd              59972
NewCasesStudents           20619
CumulativeCasesStudents    59972
dtype: int64

In [8]:
schoolCase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59972 entries, 0 to 59971
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   StateName                59972 non-null  object        
 1   StateAbbrev              59972 non-null  object        
 2   NCESDistrictID           59972 non-null  object        
 3   TimePeriodEnd            59972 non-null  datetime64[ns]
 4   NewCasesStudents         20619 non-null  Int64         
 5   CumulativeCasesStudents  59972 non-null  int32         
dtypes: Int64(1), datetime64[ns](1), int32(1), object(3)
memory usage: 2.6+ MB


In [9]:
# Sort the DataFrame by 'StateName', 'NCESDistrictID', and 'TimePeriodEnd'
schoolCase_df = schoolCase_df.sort_values(by=['StateName', 'NCESDistrictID', 'TimePeriodEnd'],ascending=[True, True, True])
schoolCase_df.reset_index(drop=True, inplace=True)
schoolCase_df.head()

,StateName,StateAbbrev,NCESDistrictID,TimePeriodEnd,NewCasesStudents,CumulativeCasesStudents
0,Arkansas,AR,5000010,2021-01-14,<NA>,26
1,Arkansas,AR,5000010,2021-01-18,<NA>,26
2,Arkansas,AR,5000010,2021-01-21,<NA>,28
3,Arkansas,AR,5000010,2021-01-25,<NA>,29
4,Arkansas,AR,5000010,2021-01-28,<NA>,33


In [10]:
# Initialize variables
prev_cumulative = schoolCase_df.iloc[0]['CumulativeCasesStudents']
new_district = True

# Iterate through the DataFrame by index
for i in range(len(schoolCase_df)):
    row = schoolCase_df.iloc[i]
    # Run initial record and append existing cases to NewCasesStudents field value
    if new_district:
        if pd.isnull(row['NewCasesStudents']):
            schoolCase_df.at[i, 'NewCasesStudents'] = row['CumulativeCasesStudents']
        new_district = False
    else:
        # From 2nd records and on when NewCasesStudents value is null will calculate from CumulativeCaseStudents value and append.
        if pd.isnull(row['NewCasesStudents']) and pd.notnull(row['CumulativeCasesStudents']):
            prev_row = schoolCase_df.iloc[i - 1]
            if row['NCESDistrictID'] == prev_row['NCESDistrictID']:
                new_cases = row['CumulativeCasesStudents'] - prev_cumulative
                schoolCase_df.at[i, 'NewCasesStudents'] = new_cases
            else:
                # If is it new NCESDistrictID will append CumulativeCasesStudents value to NewCaseStudents field.
                schoolCase_df.at[i, 'NewCasesStudents'] = row['CumulativeCasesStudents']
                new_district = False
    
    prev_cumulative = row['CumulativeCasesStudents']
    
schoolCase_df.head()

,StateName,StateAbbrev,NCESDistrictID,TimePeriodEnd,NewCasesStudents,CumulativeCasesStudents
0,Arkansas,AR,5000010,2021-01-14,26,26
1,Arkansas,AR,5000010,2021-01-18,0,26
2,Arkansas,AR,5000010,2021-01-21,2,28
3,Arkansas,AR,5000010,2021-01-25,1,29
4,Arkansas,AR,5000010,2021-01-28,4,33


In [11]:
# Define columns to keep
schoolCase_df = schoolCase_df[['StateName', 'StateAbbrev', 'NCESDistrictID', 'TimePeriodEnd', 'NewCasesStudents']]

# Rename columns in the school_enrollmentByState DataFrame
schoolCase_df.columns = ['state_name', 'state', 'DistrictID', 'TimePeriodEnd', 'CasesStudents']

In [12]:
# Group by 'StateName' and the year and month extracted from 'TimePeriodEnd'
school_group = schoolCase_df.groupby(['state_name', 'state', schoolCase_df['TimePeriodEnd'].dt.to_period('M')])['CasesStudents'].sum().reset_index()

# Rename the columns for better representation
school_group.rename(columns={'TimePeriodEnd': 'EndDate', 'CasesStudents': 'TotalCasesStudents'}, inplace=True)

# Sort the DataFrame by 'StateName' and 'YYYY-MM'
school_group.sort_values(by=['state_name', 'EndDate'], inplace=True)

# Reset index
school_group.reset_index(drop=True, inplace=True)

In [13]:
# Save to CSV file 
school_group.to_csv('clean_data/school_group.csv' , index=False)